# Zadatak 2

Izvršiti klasifikaciju tipova proteinskih sekvenci na osnovu upotrebe kodona kod SARS2
koronavirusa

- proteinske i kodirajuće sekvence preuzeti sa [NCBI Virus](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Nucleotide&VirusLineage_ss=SARS-CoV-2,%20taxid:2697049).

- upotrebu kodona preuzeti sa [The Genetic Codes](https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi).
Za SARS2 koristiti standardni kod (transl_table=1)

In [ ]:
import warnings
import pandas as pd
import numpy as np
import dtreeviz
import dalex as dx
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import ComplementNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, balanced_accuracy_score, precision_recall_curve, confusion_matrix, classification_report

from mlxtend.evaluate import RandomHoldoutSplit

from openTSNE import TSNE

from annoy import AnnoyIndex

from xgboost import XGBClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Recall

from scikeras.wrappers import KerasClassifier

from sklearn_genetic import ExponentialAdapter, GASearchCV
from sklearn_genetic.callbacks import DeltaThreshold
from sklearn_genetic.space import Categorical, Integer, Continuous

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)

stop = 5
start = 7

tous = 11
pops = 31
seed = 196883

sample = 4224
maxiter = np.iinfo(np.int32).max

t_stop = 1e-4
t_sample = 0.2

a_rate = 0.1
m_prob = 0.2
c_prob = 0.8

n_features = 7

np.random.seed(seed)

pio.renderers.default = 'notebook'

## Priprema podataka

Učitavamo podatke sa jedinstvenim kodirajućim sekvencama proteina SARS-CoV-2.

In [ ]:
df = pd.read_csv('samo_jedinstvene_kodirajuce_sekvence.csv', header=None, on_bad_lines='warn')

In [ ]:
df.shape

In [ ]:
df.head()

Izdvajamo kolonu sa kodirajućim sekvencama i kolonu sa oznakama tipova proteina.

In [ ]:
df = df[[14, 12]]
df.shape

### Čišćenje podataka

Proveravamo da li su kodirajuće sekvence odgovarajuće dužine.
One koje nisu, odbacujemo.

In [ ]:
length = np.array([len(a) for a in df[14]])
valid = (length > 8) & (length % 3 == 0)

df = df[valid]
df.shape

Proveravamo da li se kodirajuće sekvence sastoje samo od slova A, T, C i G.
Sekvence koje ne ispunjavaju ovaj uslov odbacujemo.

In [ ]:
valid = df[14].map(lambda a: set(a).issubset(set('ATCG')))

df = df[valid]
df.shape

Proveravamo koliko instanci imamo u svakoj klasi.
Odnosno koliko sekvenci imamo za svaki tip proteina.
Klase koje sadrže 64 instanci i manje odbacujemo zajedno sa instancom.

In [ ]:
proteins = df.groupby([12])[12].count()
valid = np.array([proteins[a] for a in df[12]]) > 64

df = df[valid]
df.shape

Nakon čišćenja podataka, od jedinstvenih 889737 kodirajućih ostalo nam je 869858 instanci.
Ukupno 19879 instanci je odbačeno kao nevalidno, odnosno oko $2.23\%$ skupa jedinstvenih kodirajućih sekvenci.

### Izračunavanje upotrebe kodona

Računamo koliko često neki kodon kodira svoju amino-kiselinu u svakoj kodirajućoj sekvenci.

Potrebna nam je standardna tabela kodona.

In [ ]:
standard_codon_table = {
    'TTT': 'F', 'TCT': 'S', 'TAT': 'Y', 'TGT': 'C',
    'TTC': 'F', 'TCC': 'S', 'TAC': 'Y', 'TGC': 'C',
    'TTA': 'L', 'TCA': 'S', 'TAA': 'O', 'TGA': 'O',
    'TTG': 'L', 'TCG': 'S', 'TAG': 'O', 'TGG': 'W',

    'CTT': 'L', 'CCT': 'P', 'CAT': 'H', 'CGT': 'R',
    'CTC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R',
    'CTA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R',
    'CTG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R',

    'ATT': 'I', 'ACT': 'T', 'AAT': 'N', 'AGT': 'S',
    'ATC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S',
    'ATA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R',
    'ATG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R',

    'GTT': 'V', 'GCT': 'A', 'GAT': 'D', 'GGT': 'G',
    'GTC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G',
    'GTA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G',
    'GTG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'
}

Potrebna nam je i inverzna tabela kodona.

In [ ]:
inverse_codon_table = {
    'F': ('TTT', 'TTC'),
    'L': ('CTT', 'CTC', 'CTA', 'CTG', 'TTA', 'TTG'),
    'I': ('ATT', 'ATC', 'ATA'),
    'M': ('ATG', ),
    'V': ('GTT', 'GTC', 'GTA', 'GTG'),
    'S': ('TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'),
    'P': ('CCT', 'CCC', 'CCA', 'CCG'),
    'T': ('ACT', 'ACC', 'ACA', 'ACG'),
    'A': ('GCT', 'GCC', 'GCA', 'GCG'),
    'Y': ('TAT', 'TAC'),
    'O': ('TAA', 'TGA', 'TAG'),
    'H': ('CAT', 'CAC'),
    'Q': ('CAA', 'CAG'),
    'N': ('AAT', 'AAC'),
    'K': ('AAA', 'AAG'),
    'D': ('GAT', 'GAC'),
    'E': ('GAA', 'GAG'),
    'C': ('TGT', 'TGC'),
    'W': ('TGG', ),
    'R': ('CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'),
    'G': ('GGT', 'GGC', 'GGA', 'GGG')
}

Za svaku sekvencu brojimo koliko sadrži kodona.

In [ ]:
def count_codons(seq):
    seq = seq[14]
    
    codon_counts = {a: 0 for a in standard_codon_table.keys()}
    
    for i in range(0, len(seq) - 2, 3):
        codon = seq[i:i+3]
        codon_counts[codon] += 1
    
    return codon_counts

codon_counts = df[[14]].apply(count_codons, axis=1, result_type='expand').to_dict('list')
codon_counts = {key: np.array(val, dtype='float64') for key, val in codon_counts.items()}

Za prebrojavanje amino-kiselina u prevedenim sekvencama, nećemo eksplicitno prevoditi sekvence i brojati.
Iskoristićemo činjenicu da smo već izbrojali kodone u kodirajućim sekvencama.
Sada ostaje da za svaku amino-kiselinu saberemo broj kodona koji ih kodiraju, za svaku sekvencu.

In [ ]:
length = df.shape[0]

amino_counts = {a: np.zeros(length) for a in inverse_codon_table.keys()}

for amino, codons in inverse_codon_table.items():
    for codon in codons:
        amino_counts[amino] += codon_counts[codon]

Neka se kodon *X* pojavljuje *x* puta u nekoj sekvenci i kodira amino-kiselinu *Y*.
U prevedenoj sekvenci, nakon transkripcije i translacije, amino-kiselina *Y* se pojavljuje *y* puta.
Vrednost
$$
f(X) = \frac{x}{y}
$$
predstavlja upotrebu kodona *X*.

Postoje sekvence koje ne kodiraju sve amino-kiseline, što prilikom primene formule dovodi do deljenja nulom i dodele *NaN* vrednosti.
Zamenićemo *NaN* vrednosti nulom.

In [ ]:
for codon in standard_codon_table.keys():
    codon_counts[codon] /= amino_counts[standard_codon_table[codon]]
    np.nan_to_num(codon_counts[codon], copy=False, nan=0, posinf=0, neginf=0) # ako delimo nulom

### Završna priprema podataka i podela na skupove

Rečnik sa vrednostima upotrebe kodona prevodimo u pogodniji format za rad sa tabelarnim podacima.
Odbacujemo podatke o upotrebi stop kodona.

In [ ]:
X = pd.DataFrame(codon_counts)
X.drop(['TAA', 'TGA', 'TAG', 'ATG'], axis=1, inplace=True)

y = df[12]
y.name = 'Protein'

df = X.join(y, validate='one_to_one')

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

# df.to_csv('proteini_ucestalost_kodona.csv', index=False)
df.head()

Pravimo enkoder koji kateogričkim ciljnim podacima dodeljuje celobrojne vrednosti.

In [ ]:
enc = LabelEncoder()
_ = enc.fit(y.unique())

In [ ]:
ohe = LabelBinarizer()
_ = ohe.fit(y.unique())

Konačno, delimo podatke na podatke za test i trening skupove.
Test skup će se sadržati 20% podataka, a ostatak će sadržati trening skup.
Pravimo stratifikovanu podelu, kako bi sve klase bile jednako zastupljene u test i trening skupu.
Dalje u analizi ćemo se fokusirati na trening skup.
Test skup ćemo koristiti isključivo za proveru modela.
U svim ostalim situacijama ponašamo se kao da test skup ne postoji.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)

A small training data sample that will be used for certain visualization techniques.

In [ ]:
train_sample = X_train.join(y_train, validate='one_to_one').groupby(['Protein']).sample(8, random_state=seed).reset_index(drop=True)

X_train_sample = train_sample.drop(['Protein'], axis=1)
y_train_sample = train_sample['Protein']

Long form of the small sampled training data set.

In [ ]:
shape = train_sample.shape[0] * (train_sample.shape[1] - 1)

train_sample_long = {
    'index': np.empty(shape, np.int32),
    'codon': [None for _ in range(shape)],
    'usage': np.empty(shape, np.float64),
    'protein': [None for _ in range(shape)]
}

j = 0
for i, s in X_train_sample.iterrows():
    for codon, freq in s.items():
        train_sample_long['index'][j] = i
        train_sample_long['codon'][j] = codon
        train_sample_long['usage'][j] = freq
        train_sample_long['protein'][j] = y_train_sample.iloc[i]
        j += 1

train_sample_long = pd.DataFrame(train_sample_long)

## Vizualizacija i upoznavanje sa podacima

Neuravnoteženost klasa je veoma izražena.
Najviše su zastupljene klase ORF1ab i ORF1a, kojima pripada oko $79\%$ instanci.
Više od $95\%$ instanci predstavlja pet najzastupljenijih, od ukupno dvanaest klasa.

In [ ]:
proteins = y_train.value_counts()

In [ ]:
fig = px.bar(x=proteins.index,
             y=proteins[:],
             labels={'x': 'proteins',
                     'y': 'samples'},
             text_auto=True,
             title='Class Balance',
             width=1394,
             height=450)

fig.show()

### $\chi ^2$ stats for target and features

In [ ]:
c_2, p_v = chi2(X_train, y_train)

ftc = pd.DataFrame({'chi2': c_2, 'p_value': p_v, 'codon': X_train.columns})

In [ ]:
fig = px.bar(ftc,
             x='codon',
             y=['chi2', 'p_value'],
             labels={'x': 'codons',
                     'y': 'value'},
             text_auto=True,
             color_continuous_scale= px.colors.sequential.Jet,
             title='Chi-squared stats between features and class',
             width=1394,
             height=450)

fig.show()

### Parallel Coordinates

Certainly this is an interesting looking plot.
Its usefulness leaves some doubts, though.
Difficult to see the forrest from the trees.

In [ ]:
fig = px.parallel_coordinates(X_train_sample,
                              color=enc.transform(y_train_sample),
                              color_continuous_scale=px.colors.sequential.Jet,
                              title='Parallel Coordinates',
                              width=1394,
                              height=450)

fig.update_layout(
    coloraxis_colorbar={
        'title': 'Protein',
        'tickvals': np.array(range(12)),
        'ticktext': [
            a.split(' ')[0] for a in enc.inverse_transform(
                np.array(range(12))
        )],
        'tickmode': 'array'
    }
)

fig.show()

### Radar Chart

The same thing like parallel coordinates, but a bit different.
Abstract art.
It appears that each class has different distribution and slightly different distinguishing features.

In [ ]:
fig = px.line_polar(train_sample_long,
                    r='usage',
                    theta='codon',
                    color='protein',
                    line_close=True,
                    line_group='index',
                    color_discrete_sequence=px.colors.sequential.Jet,
                    range_r=[0, 1],
                    title='Radar Chart',
                    width=1394,
                    height=450,
                    render_mode='webgl')

fig.show()

### t-SNE

Stohastičko umetanje suseda sa t-raspodelim, kraće t-SNE, je nelinearni algoritam za dimenzionu redukciju.
Važna osobina algoritma je da čuva relativna rastojanja, tako da slične podatke sa velikom verovatnoćom projektuje u tačke koje se nalaze međusobno blizu, a različite podatke daleko.
Na žalost, zbog svoje stohastičke prirode, ovaj algoritam je pogodan samo za vizualizaciju podataka.

Preporučeno je da za parametar *perplexity* biramo vrednosti između 5 i 50.
Manje vrednosti previše naglašavaju lokalne strukture u podacima, dok veće vrednosti naglašavaju globalne strukture.
Pored toga, da bi algoritam konvergirao, neophodno je da dozvolimo veliki broj iteracija.
Zbog visoke dimenzionalnosti podataka biramo *Menhetn* rastojanje.

In [ ]:
tsne = TSNE(n_components=2,
            perplexity=48,
            exaggeration=4,
            n_iter=8192,
            early_exaggeration_iter=1024,
            n_jobs=-1, metric='manhattan',
            learning_rate='auto',
            initialization='random',
            random_state=seed)

X_train_tsne = tsne.fit(X_train.to_numpy())

Klase se izdvajaju bez preklapanja, što je ohrabrujući rezultat i obećava dobru klasifikaciju.

In [ ]:
fig = px.scatter(x=X_train_tsne[:, 0],
                 y=X_train_tsne[:, 1],
                 color=y_train,
                 color_continuous_scale=px.colors.sequential.Jet,
                 labels={'x': '', 'y': ''},
                 title='t-SNE',
                 width=1394,
                 height=450,
                 render_mode='webgl')

fig.show()

## Treniranje i evaluacija modela

Za izbor modela koristićemo unakrsnu proveru sa iscrpnom pretragom.

Za sve algoritme, u unakrsnoj proveri ćemo koristiti metriku balansirane tačnosti.

Za evaluaciju modela na trening i test skupovima posmatraćemo izvaštaj sa merama preciznosti, odziva i f1 merom, normalizovane matrice konfuzije i važnost atributa za cele skupove i za pojedinačne klase.

In [ ]:
protein_labels = proteins.index
labels2display = np.array([a.split(' ', 1)[0] for a in protein_labels])

In [ ]:
mutation_adapter = ExponentialAdapter(initial_value=c_prob,
                                      end_value=m_prob,
                                      adaptive_rate=a_rate)

In [ ]:
crossover_adapter = ExponentialAdapter(initial_value=m_prob,
                                       end_value=c_prob,
                                       adaptive_rate=a_rate)

In [ ]:
stopping_condition = DeltaThreshold(threshold=t_stop,
                                    generations=stop,
                                    metric='fitness_max')

In [ ]:
early_stopping = EarlyStopping(monitor='loss',
                               min_delta=t_stop,
                               patience=stop,
                               start_from_epoch=start,
                               restore_best_weights=True)

In [ ]:
cross_validation_generator = RandomHoldoutSplit(valid_size=0.2,
                                                random_seed=seed,
                                                stratify=True)

In [ ]:
def display_precision_recall(clf, X, y_true, title):
    y_scores = clf.predict_proba(X)
    y_onehot = pd.DataFrame(ohe.transform(y_true), columns=ohe.classes_)
    
    fig = go.Figure()
    
    fig.add_shape(
        type='line',
        line={'dash': 'dash'},
        x0=0,
        x1=1,
        y0=0,
        y1=1
    )
    
    for i in range(y_scores.shape[1]):
        y_true = y_onehot.iloc[:, i]
        y_score = y_scores[:, i]

        precision, recall, _ = precision_recall_curve(y_true, y_score)
        ap_score = average_precision_score(y_true, y_score)

        name = f'{y_onehot.columns[i]} (AP={ap_score:.2f})'
        fig.add_trace(
            go.Scattergl(x=recall,
                         y=precision,
                         name=name,
                         mode='lines')
        )
    
    fig.update_layout(
        title=title,
        xaxis_title='Recall',
        yaxis_title='Precision',
        yaxis={'scaleanchor': 'x',
               'scaleratio': 1},
        xaxis={'constrain': 'domain'},
        width=1394,
        height=450
    )
    
    fig.show()

In [ ]:
def display_confusion_matrix(confusion_matrix, display_labels, title):
    fig = px.imshow(confusion_matrix.round(2),
                    labels= {'x': 'Predicted value',
                             'y': 'Real value'},
                    x=display_labels,
                    y=display_labels,
                    title=title,
                    aspect='auto',
                    text_auto=True,
                    color_continuous_scale=px.colors.sequential.Plasma,
                    width=1394,
                    height=450)
    
    fig.update_layout(coloraxis_showscale=False)
    
    fig.show()

In [ ]:
def display_best_parameters(estimators):
    print(', '.join([f'{a}: {b}' for a, b in estimators.best_params_.items()]))

### Komplementarni naivni Bajesov algoritam

Komplementarna varijacija algoritma računa verovatnoću da instanca ne pripada klasi i dodeljuje je onoj klasi sa najmanjom izračunatom verovatnoćom nepripadnosti.

U *scikit-learn* implementaciji je ugrađen i algoriam za aditivno uglađivanje podataka koji prima *alpha* parametar.

In [ ]:
params = {'alpha': Continuous(1e-3, 1e3, random_state=seed)}

In [ ]:
clf = GASearchCV(ComplementNB(),
                 param_grid=params,
                 scoring='balanced_accuracy',
                 population_size=pops,
                 generations=maxiter,
                 tournament_size=tous,
                 elitism=True,
                 crossover_probability=crossover_adapter,
                 mutation_probability=mutation_adapter,
                 cv=cross_validation_generator,
                 algorithm='eaSimple',
                 n_jobs=-1,
                 verbose=False)

_ = clf.fit(X_train,
            y_train,
            callbacks=stopping_condition)

Parametri najboljeg modela.

In [ ]:
display_best_parameters(clf)

Izveštaj za trening skup.

In [ ]:
y_true, y_pred = y_train, clf.predict(X_train)

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Matrica konfuzije za trening skup.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for ComplementNB - training data')

Precision-recall curves.

In [ ]:
display_precision_recall(clf,
                         X_train,
                         y_train,
                         'Precision-recall curves for ComplementNB - training data')

Izveštaj za test skup.

In [ ]:
y_true, y_pred = y_test, clf.predict(X_test)

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Matrica konfuzije za test skup.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for ComplementNB - test data')

Precision-recall curves.

In [ ]:
display_precision_recall(clf,
                         X_test,
                         y_test,
                         'Precision-recall curves for ComplementNB - test data')

Važnost atributa za test skup.

In [ ]:
explainer = dx.Explainer(clf.best_estimator_,
                         X_train_sample,
                         enc.transform(y_train_sample),
                         predict_function=lambda a, b: enc.transform(a.predict(b)),
                         model_type='classification',
                         label='Variable importance measures for ComplementNB',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(N=sample,
                               B=start,
                               type='shap_wrapper',
                               shap_explainer_type='KernelExplainer',
                               loss_function=lambda a, b: 1-balanced_accuracy_score(a, b),
                               processes=-1,
                               random_state=seed,
                               data=X_test,
                               silent=True)

In [ ]:
mp_clf.plot(max_display=n_features,
            title='Variable importance measures for ComplementNB')

In [ ]:
mp_clf.plot(plot_type='bar',
            max_display=n_features,
            title='Variable importance measures for ComplementNB')

### Decision Tree



In [ ]:
params = {'criterion': Categorical(['gini', 'entropy'], random_state=seed),
          'max_depth': Integer(4, 32, random_state=seed),
          'min_samples_split': Integer(128, 256, random_state=seed),
          'min_samples_leaf': Integer(32, 64, random_state=seed),
          'min_impurity_decrease': Continuous(0, 1, random_state=seed),
          'ccp_alpha': Continuous(0, 2, random_state=seed)}

In [ ]:
clf = GASearchCV(DecisionTreeClassifier(class_weight='balanced',
                                        random_state=seed),
                 param_grid=params,
                 scoring='balanced_accuracy',
                 population_size=pops,
                 generations=maxiter,
                 tournament_size=tous,
                 elitism=True,
                 crossover_probability=crossover_adapter,
                 mutation_probability=mutation_adapter,
                 cv=cross_validation_generator,
                 algorithm='eaSimple',
                 n_jobs=-1,
                 verbose=False)

_ = clf.fit(X_train,
            enc.transform(y_train),
            callbacks=stopping_condition)

Parametri najboljeg modela.

In [ ]:
display_best_parameters(clf)

Izveštaj za trening skup podataka.

In [ ]:
y_true, y_pred = y_train, enc.inverse_transform(clf.predict(X_train))

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Matrica konfuzije za trening skup podataka.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for DecisionTreeClassifier - training data')

Precision-recall curves.

In [ ]:
display_precision_recall(clf,
                         X_train,
                         y_train,
                         'Precision-recall curves for DecisionTreeClassifier - training data')

Izveštaj za test skup podataka.

In [ ]:
y_true, y_pred = y_test, enc.inverse_transform(clf.predict(X_test))

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Matrica konfuzije za test skup podataka.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for DecisionTreeClassifier - test data')

Precision-recall curves.

In [ ]:
display_precision_recall(clf,
                         X_test,
                         y_test,
                         'Precision-recall curves for DecisionTreeClassifier - test data')

Grafički prikaz drveta odlučivanja.

In [ ]:
viz_model = dtreeviz.model(clf.best_estimator_,
                           X_train=X_train.values,
                           y_train=enc.transform(y_train),
                           feature_names=X_train.columns.to_list(),
                           target_name='Proteins',
                           class_names=list(enc.inverse_transform(clf.best_estimator_.classes_)))

In [ ]:
viz_model.view(scale=0.75,
               leaftype='barh')

In [ ]:
viz_model.ctree_leaf_distributions()

In [ ]:
viz_model.ctree_feature_space(features=['TTC', 'TGC'])

Važnost atributa za test skup podataka.

In [ ]:
explainer = dx.Explainer(clf.best_estimator_,
                         X_train_sample,
                         enc.transform(y_train_sample),
                         model_type='classification',
                         label='Variable importance measures for DecisionTreeClassifier',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(N=sample,
                               B=start,
                               type='shap_wrapper',
                               shap_explainer_type='KernelExplainer',
                               loss_function=lambda a, b: 1-balanced_accuracy_score(a, b),
                               processes=-1,
                               random_state=seed,
                               data=X_test,
                               silent=True)

In [ ]:
mp_clf.plot(max_display=n_features,
            title='Variable importance measures for DecisionTreeClassifier')

In [ ]:
mp_clf.plot(plot_type='bar',
            max_display=n_features,
            title='Variable importance measures for DecisionTreeClassifier')

### Nasumična šuma

Unakrsnom proverom ćemo testirati modele sa različitim brojem klasifikatora i najvećim dozvoljenim brojevima atributa i instanci u uzorcima.
Pustićemo da se stabla preprilagode.

In [ ]:
params = {'n_estimators': Integer(32, 256, random_state=seed),
           'criterion': Categorical(['gini', 'entropy'], random_state=seed),
           'max_features': Continuous(0, 1, random_state=seed),
           'max_samples': Continuous(0, 1, random_state=seed)}

In [ ]:
clf = GASearchCV(RandomForestClassifier(n_jobs=-1,
                                        random_state=seed),
                 param_grid=params,
                 scoring='balanced_accuracy',
                 population_size=pops,
                 generations=maxiter,
                 tournament_size=tous,
                 elitism=True,
                 crossover_probability=crossover_adapter,
                 mutation_probability=mutation_adapter,
                 cv=cross_validation_generator,
                 algorithm='eaSimple',
                 n_jobs=2,
                 verbose=False)

_ = clf.fit(X_train,
            y_train,
            callbacks=stopping_condition)

Parametri najboljeg modela.

In [ ]:
display_best_parameters(clf)

Izveštaj za trening skup podataka.

In [ ]:
y_true, y_pred = y_train, clf.predict(X_train)

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Matrica konfuzije za trening skup podataka.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for RandomForestClassifier - training data')

In [ ]:
display_precision_recall(clf,
                         X_train,
                         y_train,
                         'Precision-recall curves for RandomForestClassifier - training data')

Izveštaj za test skup podataka.

In [ ]:
y_true, y_pred = y_test, clf.predict(X_test)

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Matrica konfuzije za test skup podataka.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for RandomForestClassifier - test data')

In [ ]:
display_precision_recall(clf,
                         X_test,
                         y_test,
                         'Precision-recall curves for RandomForestClassifier - test data')

Važnost atributa za test skup podataka.

In [ ]:
explainer = dx.Explainer(clf.best_estimator_,
                         X_train_sample,
                         enc.transform(y_train_sample),
                         predict_function=lambda a, b: enc.transform(a.predict(b)),
                         model_type='classification',
                         label='Variable importance measures for RandomForestClassifier',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(N=sample,
                               B=start,
                               type='shap_wrapper',
                               shap_explainer_type='KernelExplainer',
                               loss_function=lambda a, b: 1-balanced_accuracy_score(a, b),
                               processes=-1,
                               random_state=seed,
                               data=X_test,
                               silent=True)

In [ ]:
mp_clf.plot(max_display=n_features,
            title='Variable importance measures for SVC')

In [ ]:
mp_clf.plot(plot_type='bar',
            max_display=n_features,
            title='Variable importance measures for SVC')

### Extreme gradient boosting

In [ ]:
params = {'n_estimators': Integer(4, 96, random_state=seed),
           'max_depth': Integer(1, 4, random_state=seed),
           'grow_policy': Categorical([0, 1], random_state=seed),
           'gamma': Continuous(0, 1, random_state=seed),
           'learning_rate': Continuous(0, 1, random_state=seed),
           'objective': Categorical(['reg:linear',
                                     'reg:logistic',
                                     'multi:softmax',
                                     'multi:softprob'], random_state=seed)}

In [ ]:
clf = GASearchCV(XGBClassifier(nthread=-1,
                               random_state=seed),
                 param_grid=params,
                 scoring='balanced_accuracy',
                 population_size=pops,
                 generations=maxiter,
                 tournament_size=tous,
                 elitism=True,
                 crossover_probability=crossover_adapter,
                 mutation_probability=mutation_adapter,
                 cv=cross_validation_generator,
                 algorithm='eaSimple',
                 n_jobs=2,
                 verbose=False)

_ = clf.fit(X_train,
            enc.transform(y_train),
            callbacks=stopping_condition)

Parametri najboljeg modela.

In [ ]:
display_best_parameters(clf)

Izveštaj za trening skup podataka.

In [ ]:
y_true, y_pred = y_train, enc.inverse_transform(clf.predict(X_train))

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Matrica konfuzije za trening skup podataka.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for XGBClassifier - training data')

In [ ]:
display_precision_recall(clf,
                         X_train,
                         y_train,
                         'Precision-recall curves for XGBClassifier - training data')

Izveštaj za test skup podataka.

In [ ]:
y_true, y_pred = y_test, enc.inverse_transform(clf.predict(X_test))

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Matrica konfuzije za test skup podataka.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for XGBClassifier - training data')

In [ ]:
display_precision_recall(clf,
                         X_test,
                         y_test,
                         'Precision-recall curves for XGBClassifier - test data')

Važnost atributa za test skup podataka.

In [ ]:
explainer = dx.Explainer(clf.best_estimator_,
                         X_train_sample,
                         enc.transform(y_train_sample),
                         model_type='classification',
                         label='Variable importance measures for XGBClassifier',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(N=sample,
                               B=start,
                               type='shap_wrapper',
                               shap_explainer_type='KernelExplainer',
                               loss_function=lambda a, b: 1-balanced_accuracy_score(a, b),
                               processes=-1,
                               random_state=seed,
                               data=X_test,
                               silent=True)

In [ ]:
mp_clf.plot(max_display=n_features,
            title='Variable importance measures for XGBClassifier')

In [ ]:
mp_clf.plot(plot_type='bar',
            max_display=n_features,
            title='Variable importance measures for XGBClassifier')

### Linear Support Vector Machine

Parameters for cross-validation.

In [ ]:
params = {'C': Continuous(0, 1e3, random_state=seed),
          'intercept_scaling': Continuous(0, 1e3, random_state=seed)}

Applying genetic search cross-validation algorithm.

In [ ]:
clf = GASearchCV(LinearSVC(dual='auto',
                           class_weight='balanced',
                           tol=t_stop,
                           max_iter=maxiter,
                           random_state=seed),
                 param_grid=params,
                 scoring='balanced_accuracy',
                 population_size=pops,
                 generations=maxiter,
                 tournament_size=tous,
                 elitism=True,
                 crossover_probability=crossover_adapter,
                 mutation_probability=mutation_adapter,
                 cv=cross_validation_generator,
                 algorithm='eaSimple',
                 n_jobs=-1,
                 verbose=False)

_ = clf.fit(X_train,
            y_train,
            callbacks=stopping_condition)

Parameters for the best model.

In [ ]:
display_best_parameters(clf)

Probability calibration with isotonic regression.

In [ ]:
clf = CalibratedClassifierCV(clf.best_estimator_,
                             method='isotonic',
                             cv='prefit')

_ = clf.fit(X_train,
            y_train)

Classification report for training data.

In [ ]:
y_true, y_pred = y_train, clf.predict(X_train)

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Confusion matrix for training data.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for LinearSVC - training data')

Precision-recall curves for training data.

In [ ]:
display_precision_recall(clf,
                         X_train,
                         y_train,
                         'Precision-recall curves for LinearSVC - training data')

Classification report for test data.

In [ ]:
y_true, y_pred = y_test, clf.predict(X_test)

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Confusion matrix for test data.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for LinearSVC - test data')

Precision-recall curves for test data.

In [ ]:
display_precision_recall(clf,
                         X_test,
                         y_test,
                         'Precision-recall curves for LinearSVC - test data')

Variable importances.

In [ ]:
explainer = dx.Explainer(clf,
                         X_train_sample,
                         enc.transform(y_train_sample),
                         predict_function=lambda a, b: enc.transform(a.predict(b)),
                         model_type='classification',
                         label='Variable importance measures for LinearSVC',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(N=sample,
                               B=start,
                               type='shap_wrapper',
                               shap_explainer_type='KernelExplainer',
                               loss_function=lambda a, b: 1-balanced_accuracy_score(a, b),
                               processes=-1,
                               random_state=seed,
                               data=X_test,
                               silent=True)

In [ ]:
mp_clf.plot(max_display=n_features,
            title='Variable importance measures for LinearSVC')

In [ ]:
mp_clf.plot(plot_type='bar',
            max_display=n_features,
            title='Variable importance measures for LinearSVC')

### Artificial Neural Networks

I found a wrapper library *scikeras* that gives *tensorflow*/*keras* interface like *scikit-learn* API.
It turned out to be a bit of disappointment.

In [ ]:
def get_model(hidden_layer_dim,
              dropout_rate,
              activation_function,
              meta):
    model = Sequential()
    model.add(Dense(meta['n_features_in_'],
                    input_shape=meta['X_shape_'][1:],
                    activation=activation_function))
    model.add(Dropout(dropout_rate,
                      seed=seed))
    model.add(Dense(hidden_layer_dim,
                    activation=activation_function))
    model.add(Dropout(dropout_rate,
                      seed=seed))
    model.add(Dense(hidden_layer_dim,
                    activation=activation_function))
    model.add(Dropout(dropout_rate,
                      seed=seed))
    model.add(Dense(meta['n_classes_'],
                    activation='softmax'))
    
    return model

Parameters for the cross-valudation.

In [ ]:
params = {'optimizer': ['adam',
                        'rmsprop'],
          'loss': ['categorical_hinge',
                   'categorical_crossentropy'],
          'activation_function': ['relu',
                                  'sigmoid']}

*KerasClassifier* did not work well with the genetic algorithm, and it does not appear to be working that well with the grid search either.
The wrapper appears to be broken, as it throws warnings both left and right.
It also fails to report the score.
I have decided to keep it because I like the idea of having a decent neural network classifier with *scikit-learn* interface.

In [ ]:
clf = GridSearchCV(KerasClassifier(get_model,
                                   dropout_rate=0.2,
                                   hidden_layer_dim=31,
                                   activation_function='relu',
                                   # class_weight='balanced', # causes warnings
                                   metrics=AUC(name='prc', curve='PR'),
                                   epochs=maxiter,
                                   callbacks=early_stopping,
                                   validation_split=0.2,
                                   batch_size=-1,
                                   verbose=0,
                                   random_state=seed),
                   param_grid=params,
                   scoring=lambda a, b: balanced_accuracy_score(ohe.inverse_transform(a),
                                                                ohe.inverse_transform(b)),
                   cv=cross_validation_generator,
                   n_jobs=2)

_ = clf.fit(X_train,
            ohe.transform(y_train),
            callbacks=stopping_condition)

Parameters for the best model.

In [ ]:
display_best_parameters(clf)

Classification report for training data.

In [ ]:
y_true, y_pred = y_train, ohe.inverse_transform(clf.predict(X_train))

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Confusion matrix for training data.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for KerasClassifier - training data')

Precission-recall curves for training data.

In [ ]:
display_precision_recall(clf,
                X_train,
                y_train,
                'Precision-recall curves for KerasClassifier - training data')

Classification report for test data.

In [ ]:
y_true, y_pred = y_test, ohe.inverse_transform(clf.predict(X_test))

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Confusion matrix for test data.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for KerasClassifier - test data')

Precision-recall curves for test data.

In [ ]:
display_precision_recall(clf,
                X_test,
                y_test,
                'Precision-recall curves for KerasClassifier - test data')

Feature importances.

In [ ]:
explainer = dx.Explainer(clf.best_estimator_,
                         X_train_sample,
                         enc.transform(y_train_sample),
                         predict_function=lambda a, b: enc.transform(ohe.inverse_transform(a.predict(b))),
                         model_type='classification',
                         label='Variable importance measures for KerasClassifier',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(N=sample,
                               B=start,
                               type='shap_wrapper',
                               shap_explainer_type='KernelExplainer',
                               loss_function=lambda a, b: 1-balanced_accuracy_score(a, b),
                               processes=-1,
                               random_state=seed,
                               data=X_test,
                               silent=True)

In [ ]:
mp_clf.plot(max_display=n_features,
            title='Variable importance measures for KerasClassifier')

In [ ]:
mp_clf.plot(plot_type='bar',
            max_display=n_features,
            title='Variable importance measures for KerasClassifier')

### Aproximate k-Nearest Neighbors

A more efficient implementation of k-Nearest Neighbors search than the one given in *scikit-learn*.
Initially, I tried various implementations based on HSNW search, but I could not find a library that builds indexer deterministically.
*Annoy* indexer might be a bit slower, but at least every time I retrain the classifier, with the same set of parameters, I get the same result.

In [ ]:
class AnnoyKNN:
    _estimator_type = "classifier"
    
    def __init__(self,
                 weighted=False,
                 metric='manhattan',
                 k=5,
                 n_trees=128,
                 n_jobs=1,
                 random_state=None):
        assert isinstance(weighted, bool), 'weighted has to be bool'
        assert metric in ('angular',
                          'euclidean',
                          'manhattan',
                          'hamming',
                          'dot'), 'metric can be "angular", "euclidean", "manhattan", "hamming", or "dot"'
        assert isinstance(k, int), 'k has to be int'
        assert isinstance(n_trees, int), 'n_trees has to be int'
        assert isinstance(n_jobs, int), 'n_jobs has to be int'
        assert isinstance(random_state, (int, type(None))), 'random_state has to be int or None'
        
        self.index = None
        self.X = pd.DataFrame()
        self.y = None
        self.classes_ = None
        self.k = k
        self.metric = metric
        self.n_trees = n_trees
        self.n_jobs = n_jobs
        self.weighted = weighted
        self.random_state = random_state

    def __sklearn_clone__(self):
        clone = AnnoyKNN(self.weighted,
                         self.metric,
                         self.k,
                         self.n_trees,
                         self.n_jobs,
                         self.random_state)
        clone.X = self.X
        clone.index = self.index
        
        return clone
    
    def get_params(self, deep=True):
        return {'index': self.index,
                'y': self.y,
                'k': self.k,
                'classes_': self.classes_,
                'metric': self.metric,
                'n_trees': self.n_trees,
                'n_jobs': self.n_jobs,
                'weighted': self.weighted,
                'random_state': self.random_state}
    
    def set_params(self, **params):
        for key, val in params.items():
            if key in ('metric',
                       'n_trees',
                       'random_state'):
                self.index = None
            self.__dict__[key] = val

        assert isinstance(self.weighted, bool), 'weighted has to be bool'
        assert self.metric in ('angular',
                               'euclidean',
                               'manhattan',
                               'hamming',
                               'dot'), 'metric can be "angular", "euclidean", "manhattan", "hamming", or "dot"'
        assert isinstance(self.k, int), 'k has to be int'
        assert isinstance(self.n_trees, int), 'n_trees has to be int'
        assert isinstance(self.n_jobs, int), 'n_jobs has to be int'
        assert isinstance(self.random_state, (int, type(None))), 'random_state has to be int or None'
        assert isinstance(self.X, pd.DataFrame), 'X has to be pandas DataFrame object'
        
        return self

    def fit(self, X, y):
        assert isinstance(X, pd.DataFrame), 'X has to be pandas DataFrame object'
        if self.index is None or not X.equals(self.X):
            self.X = X
            
            self.index = AnnoyIndex(X.shape[1], self.metric)
            self.index.set_seed(self.random_state)
            
            for i, v in enumerate(X.to_numpy()):
                self.index.add_item(i, v)
            self.index.build(self.n_trees, self.n_jobs)
            
        assert isinstance(y, pd.Series), 'y has to be pandas Series object'
        self.y = y.to_numpy()
        self.classes_ = np.unique(self.y)
        
        return self

    def score(self. X, y, sample_weight=None):
        assert self.index is not None, 'first to fit the model you have'
        assert isinstance(X, pd.DataFrame), 'X has to be pandas DataFrame object'
        assert isinstance(y, (pd.Series, np.array)), 'y has to be pandas Series or numpy array object'
        y_true = y
        y_pred = self.predict(X)
        
        return balanced_accuracy_score(y_true,
                                       y_pred,
                                       sample_weight)

    def __get_weighted_votes(self, X):
        indices = np.empty((X.shape[0], self.k),
                           dtype=np.int32)
        distances = np.empty((X.shape[0], self.k),
                             dtype=np.float64)
        
        # Annoy is annoying for not having batch query ...
        for i, v in enumerate(X.to_numpy()):
            tmp_i, tmp_d = self.index.get_nns_by_vector(v,
                                                        self.k,
                                                        include_distances=self.weighted)
            indices[i] = np.array(tmp_i,
                                  dtype=np.int32)
            distances[i] = np.array(tmp_d,
                                    dtype=np.float64)
        
        vote = self.y[indices]
        weight = np.reciprocal(distances,
                               where=distances!=0,
                               out=np.zeros_like(distances))
        weighted_votes = [(vote[a], weight[a]) for a in range(indices.shape[0])]
        
        return weighted_votes
    
    def __get_unweighted_votes(self, X):        
        indices = np.empty((X.shape[0], self.k),
                           dtype=np.int32)
        
        # Annoy is annoying for not having batch query ...
        for i, v in enumerate(X.to_numpy()):
            tmp_i = self.index.get_nns_by_vector(v,
                                                 self.k,
                                                 include_distances=self.weighted)
            indices[i] = np.array(tmp_i,
                                  dtype=np.int32)
        
        unweighted_votes = self.y[indices]
        
        return unweighted_votes

    def predict(self, X):
        assert isinstance(X, pd.DataFrame), 'X has to be pandas DataFrame object'
        predictions = None
        
        if self.weighted:
            votes = self.__get_weighted_votes(X)
            
            predictions = np.array([np.argmax(np.bincount(a,
                                                          weights=b)) for a, b in votes],
                                   dtype=np.int32)
        else:
            votes = self.__get_unweighted_votes(X)
            
            predictions = np.array([np.argmax(np.bincount(a)) for a in votes],
                                   dtype=np.int32)
            
        return predictions
    
    def predict_proba(self, X):
        assert isinstance(X, pd.DataFrame), 'X has to be pandas DataFrame object'
        probabilities = None
        
        if self.weighted:
            votes = self.__get_weighted_votes(X)
            
            numerator = np.array([np.bincount(a,
                                              weights=b,
                                              minlength=self.classes_.shape[0]) for a, b in votes])
            denominator = np.array([a.sum() for a in numerator])
            probabilities = np.array([a / b for a, b in zip(numerator, denominator)])
        else:
            votes = self.__get_unweighted_votes(X)

            numerator = np.array([np.bincount(a,
                                              minlength=self.classes_.shape[0]) for a in votes])
            denominator = self.k
            probabilities = numerator / denominator
            
        return probabilities

Parameters for cross-validation.

In [ ]:
params = [{'k': [5, 13, 23, 31],
          'weighted': [True, False]}]

To achieve the best performance, since the search space is small, instead of the genetic algorithm, simple grid search will be used.

In [ ]:
clf = GridSearchCV(AnnoyKNN(n_jobs=-1,
                            random_state=seed),
                   param_grid=params,
                   scoring='balanced_accuracy',
                   cv=cross_validation_generator)

_ = clf.fit(X_train,
            pd.Series(enc.transform(y_train)))

Parameters of the best model.

In [ ]:
display_best_parameters(clf)

Classification report for training data.

In [ ]:
y_true, y_pred = y_train, enc.inverse_transform(clf.predict(X_train))

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Confusion matrix for training data.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for AnnoyKNN - training data')

Precision-recall curves for training data.

In [ ]:
display_precision_recall(clf,
                         X_train,
                         y_train,
                         'Precision-recall curves for AnnoyKNN - training data')

Classification report for test data.

In [ ]:
y_true, y_pred = y_test, enc.inverse_transform(clf.predict(X_test))

print(classification_report(y_true,
                            y_pred,
                            digits=5,
                            zero_division=0.0))

Confusion matrix for test data.

In [ ]:
cm = confusion_matrix(y_true,
                      y_pred,
                      normalize='true',
                      labels=protein_labels)

display_confusion_matrix(cm,
                         display_labels=labels2display,
                         title='Confusion matrix for AnnoyKNN - test data')

Precision-recall curves for test data.

In [ ]:
display_precision_recall(clf,
                         X_test,
                         y_test,
                         'Precision-recall curves for AnnoyKNN - test data')

Feature importances.
Since SHAP library dislikes my implementation of AnnoyKNN, and since it's so well documented that it makes debugging impossible, instead of using SHAP values, simple permutation importances will be calculated.

In [ ]:
explainer = dx.Explainer(clf.best_estimator_,
                         X_test,
                         enc.transform(y_test),
                         precalculate=False,
                         label='Variable importance measures for AnnoyKNN',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(keep_raw_permutations=False,
                               random_state=seed)

In [ ]:
mp_clf.plot(max_vars=n_features,
            title='Variable importance measures for AnnoyKNN')

## Conclusion

After loading the data, features containing coding sequences and protein types have been extracted.
Then, we cleaned up the data by dropping the sequences with inappropriate number of nucleotides, with ambiguities, or belong to the classes represented by too few instances for reliable generalization.

The preparation phase has been completed by calculating codon usage frequencies and splitting the data into train and test sets.
A couple of other utilities have been prepared, before moving on to visualization phase.

During the visualization phase, we first looked at some general statistics.
That is, distribution of instances in different classes and $\chi ^2$ test to determine feature importances.
Then we looked at parallel coordinates and radar chart of a sampled data, and finally we looked at *t-SNE* projection of the training dataset.
Then, we moved on to classification.

It appears that the data is linearly separable, as the linear-like classifiers such as *LinearSVC* and *DecisionTreeClassifier* achieved excellent results.
Even my custom built *AnnoyKNN* classifier, based on *Annoy* indexer, favored fewer neighbors and ignored distances, which implies that it prefers more linear separation of the data.

*ComplementaryNB* struggled with imbalanced data.
It simply failed to recognize the underrepresented classes.

The best results have been achieved with ensemble methods based on decision trees.
That doesn't come as a surprise, as even a single decision tree has achieved significant result.

This version of *KerasClassifier* appears to be broken.
In conjunction with *GASearchCV* it fails to report classification score, which is necessary for triggering early stopping callback.
Instead it returned NaN values.
When used with ordinary *GridSearchCV* it triggerd a lot of warnings for variables that are even not accessible from *scikeras* API.
Finally, the model it generated is practically useless.

This version of *GASearchCV* does not parallelize at all, despite having n_jobs parameter.
If it has to evaluate *n* models during an iteration, and it has *n* cores available, I would expect that it would train each model in a separate process.
Instead, the models are trained one at a time, which is a serious flaw of this implementation.

Overall, we trained seven models, out of which five have demonstrated excellent performance.